# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

- Face Detection
    - 이미지나 동영상에서 사람의 얼굴 특징을 찾는것 



In [1]:
import mediapipe as mp

In [1]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection         # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils                 # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

cap = cv2.VideoCapture(0)

# 이미지 불러오기
image_right_eye = cv2.imread("./right_eye.png")
image_left_eye = cv2.imread("./left_eye.png")
image_nose = cv2.imread("./nose.png")

# model_selection : face-dection 객체를 만들 때 상황(0 : 1 = 근거리영상 : 원거리영상 )에 맞는 영상에 따라 선택
# min_detection_confidence : 얼굴로 판단할 어느정도의 신뢰도를 정의 (treshold와 비슷한 개념)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)                 # process : 이미지로부터 얼굴을 검출해 결과를 반환하는 함수 

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        image = cv2.resize(image, (1920,1080))
        
        if results.detections:
            # 6개의 특징을 추출 : 우좌/눈, 코 끝, 입중심, 우좌/귀( 귀구슬점, 이주 )
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)     # 얼굴이라고 인식하는 6부분에 점을 찍기
                
                # 특정 위치 가져오기 ( 상대 위치임 )
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]                        # 오른쪽 눈
                left_eye = keypoints[1]                         # 왼쪽 눈 
                nose_tip = keypoints[2]                         # 코 끝부분
                h, w, _ = image.shape                           # 이미지의 height, width, channel 3개의 정보 반환 
                
                right_eye = (int(right_eye.x * w), int(right_eye.y * h)-100)        # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h)-100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 각 특징에 이미지 그리기
                # [오른쪽 눈 높이 설정 , 오른쪽 눈 너비 설정]  -- > [높이 100 , 너비 100]
                image[right_eye[1]-50 : right_eye[1]+50, right_eye[0]-50 : right_eye[0]+50] = image_right_eye
                image[left_eye[1]-50 : left_eye[1]+50, left_eye[0]-50 : left_eye[0]+50] = image_left_eye
                image[nose_tip[1]-50 : nose_tip[1]+50, nose_tip[0]-150 : nose_tip[0]+150] = image_nose
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy =0.5))
        if cv2.waitKey(5) == ord('q'):
            print(image.shape)
            break
cap.release()
cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (100,300,3) into shape (98,300,3)

: 


- Face Recognition
    - 이미지나 동영상에서 누구의 얼굴인지 찾는것

### 추가학습

- <a href = "https://opencv-python.readthedocs.io/en/latest/">openCV 추가학습 페이지</a>